In [2]:
# Deep Learning with Python Ch8: style transfer
############################
#   neural style transfer  #
############################
# load images
from keras.preprocessing.image import load_img, img_to_array

# location of the image you want to transform
target_img_path = 'C:\\Users\\Carol\\Desktop\\temple.jpg'
# location of the image that has the style you want to transform to
style_img_path  = 'C:\\Users\\Carol\\Desktop\\mediterranean_landscape_picasso.jpg'

# dimensions of the transformed image
width, height = load_img(target_img_path).size
img_height    = 400
img_width     = int(width * img_height / height)

In [3]:
# auxiliary fns
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# reverse transformations done by vgg19.preprocess_input
def deprocess_image(x):
    # zero-centering by removing the mean pixel value from ImageNet
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.680
    # convert image from BGR back to RGB
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [5]:
# load pretrained VGG19 network & apply to the 3 images
from keras import backend as K

target_img = K.constant(preprocess_image(target_img_path))
style_img  = K.constant(preprocess_image(style_img_path))
combo_img  = K.placeholder((1, img_height, img_width, 3)) # this will contain the transformed image

input_tensor = K.concatenate([target_img, style_img, combo_img], axis=0)

model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('model loaded')

model loaded


In [6]:
# define 3 lose components
# content loss
def content_loss(base, combo):
    return K.sum(K.square(combo-base))

# style loss
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2,0,1)))
    gram     = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combo):
    S        = gram_matrix(style)
    C        = gram_matrix(combo)
    channels = 3
    size     = img_height * img_width
    return K.sum(K.square(S-C)) / (4 * (channels**2) * (size **2))

# total variation loss
def variation_loss(x):
    a = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a+b, 1.25))

In [7]:
# define the final loss fn
outputs_dict  = dict([(layer.name, layer.output) for layer in model.layers])
# layer used for content loss
content_layer = 'block5_conv2'
# layers used for style loss
style_layers  = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

# weight for each loss component
content_wt    = 0.025
style_wt      = 1
variation_wt  = 1e-4

# initiate the final loss as a scalar
loss = K.variable(0)

# add conent loss
layer_features      = outputs_dict[content_layer] 
target_img_features = layer_features[0, :, :, :]
combo_features      = layer_features[2, :, :, :]
loss               += content_wt * content_loss(target_img_features, combo_features)

# add style loss
for layer_name in style_layers:
    layer_features     = outputs_dict[layer_name]
    style_img_features = layer_features[1, :, :, :]
    combo_features     = layer_features[2, :, :, :] 
    loss              += (style_wt / len(style_layers)) * style_loss(style_img_features, combo_features)

# add total variation loss
loss += variation_wt * variation_loss(combo_img)

In [8]:
# set up gradient descent process
# get gradients of the transformed image w.r.t. loss
grads = K.gradients(loss, combo_img)[0]

# fn to fetch the values of both current loss and current gradients
fetch_loss_and_grads = K.function([combo_img], [loss, grads])

# create a class that wraps fetch_loss_and_grads in a way that
# lets us retrieve loss and gradients via 2 seperate method calls, 
# which is required by the SciPy optimizer
class Evaluator(object):
    def __init__(self):
        self.loss_value   = None
        self.grads_values = None
    
    def loss(self, x):
        assert self.loss_value is None
        x                 = x.reshape((1, img_height, img_width, 3))
        outs              = fetch_loss_and_grads([x])
        loss_value        = outs[0]
        grads_values      = outs[1].flatten().astype('float64')
        self.loss_value   = loss_value
        self.grads_values = grads_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grads_values      = np.copy(self.grads_values)
        self.loss_value   = None
        self.grads_values = None
        return grads_values

evaluator = Evaluator()    

In [9]:
# style transfer loop
import imageio
import time
from scipy.optimize import fmin_l_bfgs_b

result_prefix = 'my_result'
iterations    = 20

x = preprocess_image(target_img_path)
x = x.flatten()

for i in range(iterations):
    print('start itertaion', i)
    start_time = time.time()
    
    # run L-BFGS optimization over the pixels of the transformed image to minimize loss
    # you must pass the fn that computes loss and the fn that computes gradients as 2 seperate arguments
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('current loss value:', min_val)
    
    # save transformed image per iteration
    img   = x.copy().reshape((img_height, img_width, 3))
    img   = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' %i
    imageio.imwrite(fname, img)
    print('image saved as', fname)
    
    end_time = time.time()
    print('iteration %d completed in %ds' %(i, end_time-start_time))

start itertaion 0
current loss value: 4980140000.0
image saved as my_result_at_iteration_0.png
iteration 0 completed in 30s
start itertaion 1
current loss value: 1668581200.0
image saved as my_result_at_iteration_1.png
iteration 1 completed in 24s
start itertaion 2
current loss value: 918632260.0
image saved as my_result_at_iteration_2.png
iteration 2 completed in 27s
start itertaion 3
current loss value: 666354400.0
image saved as my_result_at_iteration_3.png
iteration 3 completed in 29s
start itertaion 4
current loss value: 532935650.0
image saved as my_result_at_iteration_4.png
iteration 4 completed in 31s
start itertaion 5
current loss value: 441596450.0
image saved as my_result_at_iteration_5.png
iteration 5 completed in 33s
start itertaion 6
current loss value: 381881570.0
image saved as my_result_at_iteration_6.png
iteration 6 completed in 31s
start itertaion 7
current loss value: 344760600.0
image saved as my_result_at_iteration_7.png
iteration 7 completed in 37s
start itertaio